In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm 

In [2]:
layer_0_sent_out = np.load('/depot/jgmakin/data/auditory_cortex/opt_inputs/conv_layer_0.npy', allow_pickle=True)
layer_1_sent_out = np.load('/depot/jgmakin/data/auditory_cortex/opt_inputs/conv_layer_1.npy', allow_pickle=True)

layer_0_opt_out = np.load('/depot/jgmakin/data/auditory_cortex/opt_inputs/conv_layer_0_outputs.npy', allow_pickle=True)
layer_1_opt_out = np.load('/depot/jgmakin/data/auditory_cortex/opt_inputs/conv_layer_1_outputs.npy', allow_pickle=True)

spikes_20 = np.load('/depot/jgmakin/data/auditory_cortex/spikes/win_20.npy', allow_pickle=True)
spikes_40 = np.load('/depot/jgmakin/data/auditory_cortex/spikes/win_40.npy', allow_pickle=True)


In [3]:
sent = 0
spikes_40_sent_0 = np.concatenate([s.reshape(1, -1) for s in spikes_40[sent]], axis=0)
spikes_20_sent_0 = np.concatenate([s.reshape(1, -1) for s in spikes_20[sent]], axis=0)

In [4]:
from transformers import Speech2TextFeatureExtractor, Speech2TextModel
from scipy.io.wavfile import read

from optimal_input.get_values import GetValues


model = Speech2TextModel.from_pretrained("facebook/s2t-small-librispeech-asr")
feature_extractor = Speech2TextFeatureExtractor.from_pretrained("facebook/s2t-small-librispeech-asr")
# reg = transformer_regression(dir, subject)

for param in model.parameters():
    param.requires_grad = False

layers = ["encoder.conv.conv_layers.0","encoder.conv.conv_layers.1","encoder.layers.0.fc2",
			"encoder.layers.1.fc2","encoder.layers.2.fc2","encoder.layers.3.fc2",
			"encoder.layers.4.fc2","encoder.layers.5.fc2","encoder.layers.6.fc2",
			"encoder.layers.7.fc2","encoder.layers.8.fc2","encoder.layers.9.fc2"]


get_layer_output = GetValues(model)
l = layers[1]
layer_outs = []
for l in layers[:2]:
# for i in range(1):
    # print(i)
    sr, aud = read("/depot/jgmakin/data/audio_data/sent_" + str(0) +".wav")
    spect = feature_extractor(aud, padding=False, sampling_rate=16000, return_tensors="pt").input_features
    get_layer_output.get_layer_output(spect, l)

    layer_outs.append(get_layer_output.hook.output_f[0].detach().numpy())

/home/akamsali/anaconda3/envs/research/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at facebook/s2t-small-librispeech-asr were not used when initializing Speech2TextModel: ['lm_head.weight']
- This IS expected if you are initializing Speech2TextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Speech2TextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Speech2TextModel were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr

In [75]:
a = np.zeros((layer_outs[0].shape[0], layer_outs[0].shape[1]//2))
k=2
for i in range(layer_outs[0].shape[1]//2):
    # print(k*i, k*(i+1))
    a[:, i] = layer_outs[0][:, k*i:k*(i+1)].sum(axis=1)

down_sample_layer_0 = np.concatenate((a, layer_outs[0][:,-1].reshape(-1,1)), axis=1)


In [68]:
betas_layer_0 = np.linalg.solve(down_sample_layer_0.dot(down_sample_layer_0.T), (down_sample_layer_0).dot(spikes_40_sent_0.T))
betas_layer_1 = np.linalg.solve(layer_outs[1].dot(layer_outs[1].T), (layer_outs[1]).dot(spikes_40_sent_0.T))

In [85]:
a = np.load('/depot/jgmakin/data/auditory_cortex/betas/layer_0.npy', allow_pickle=True)
b = np.load('/depot/jgmakin/data/auditory_cortex/betas/layer_1.npy', allow_pickle=True)

In [87]:
# layer_outs[0]
(a.T.dot(layer_outs[0]))

(64, 95)